In [1]:
import os
from pathlib import Path


import matplotlib.pyplot as plt

import MDAnalysis as mda
import numpy as np
import openmm

import openmm.app as openmm_app
import pandas as pd

import pydiffusion

from MDAnalysis.analysis import msd, rms

from scipy.optimize import curve_fit

from scipy.stats import linregress
from tools import base, simulator

d:\Programs\Envs\HomeModel2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sim = simulator.Simulation(T=20)
sim.load('systems/dmag_22_1050')
sim.equilibrate('model',steps=10)
sim.start_product('model3')

NVT/NPT minimisation: 100%|██████████| 10/10 [00:28<00:00,  2.86s/it]


In [10]:
sim.simulate(100)

Product: 100%|██████████| 100/100 [02:54<00:00,  1.75s/it]


In [4]:
sim.get_msd().to_csv('model/msd.csv')

# Plots

In [2]:
%matplotlib qt

In [5]:
u = mda.Universe('results/dmag_35_1029/15/Eq.pdb', 'results/dmag_35_1029/15/traj.xtc')

In [6]:
msd_but = pd.read_csv('results/dmag_35_1029/15/msd.csv',index_col=0)
r_but = rms.RMSD(u, u, 'resname BUT')
r_but.run()
df_but = pd.DataFrame(r_but.results.rmsd, columns=['Frame', 'Time', 'BUT'])

fig, ax = plt.subplots()
ax.plot(df_but['Time'], df_but['BUT'], label='RMSD')
ax.plot(msd_but.index,msd_but['BUT']*10**14,label='MSD')

ax.set_ylabel('RMSD (A)')
ax.legend()

In [7]:
def get_msd(universe, compound: base.MolNames, dt=1):
    selection = f'resname {compound.value}'
    m = msd.EinsteinMSD(universe, select=selection)
    m.run()
    return m.times, np.mean(m.results['msds_by_particle'], axis=1)


def analyse_lin(time, data, dt=1):
    statistics = {}
    (
        slope,
        intercept,
        statistics['R'],
        statistics['P'],
        statistics['err'],
    ) = linregress(time, data)
    D = slope / 6
    D0 = intercept
    return D, D0, statistics

In [8]:
time,y = get_msd(u,base.MolNames.dmag)

100%|██████████| 1820/1820 [00:00<00:00, 2723.92it/s]


In [9]:
D, D0,  statistics=analyse_lin(time,y)
D,statistics

(0.023231874427253062,
 {'R': 0.931408255051853,
  'P': 9.958007710672781e-221,
  'err': 0.0024409209513457392})

In [10]:
# Plot
plt.scatter(time, y, label='MSD (Average)',s=2)
plt.plot(time, 6*D * time + D0, label='Linear',c='r')
plt.xlabel('Time')
plt.ylabel('MSD')
plt.legend()
print(f"D={D: .3e} | R2={statistics['R']: .3e}")
plt.show()

D= 2.323e-02 | R2= 9.314e-01


In [11]:
plt.scatter(time, y/y.mean(), label='MSD (Average)',s=2,c='r',)
plt.plot(msd_but.index,msd_but['BUT']/msd_but['BUT'].mean(),label='MSD',c='b')
plt.plot(time, (6*D * time + D0)/(6*D * time + D0).mean(), label='Linear',c='g')
plt.legend()